# Mesh Analysis

PolyMesh provides some useful classes and algorithms for mesh management and operations related to the geometry or the topology of polygonal meshes.

## Nodal Adjacency

In [1]:
import pyvista as pv
import numpy as np
from polymesh import PolyData

d, h, a = 6.0, 15.0, 15.0
cyl = pv.CylinderStructured(center=(0.0, 0.0, 0.0), direction=(0.0, 0.0, 1.0),
                            radius=np.linspace(d/2, a/2, 15), height=h, 
                            theta_resolution=100, z_resolution=40)
pd = PolyData.from_pv(cyl)

To access the nodal adjacency matrix as a scipy sparse matrix:

In [14]:
pd.nodal_adjacency_matrix(frmt='scipy-csr')

<60000x60000 sparse matrix of type '<class 'numpy.intc'>'
	with 1512052 stored elements in Compressed Sparse Row format>

as a NetworkX graph:

In [3]:
pd.nodal_adjacency_matrix(frmt='nx')

as an `Awkward` array:

In [4]:
pd.nodal_adjacency_matrix(frmt='jagged')

<Array [[0, 1, 15, ... 59984, 59998, 59999]] type='60000 * var * int64'>

or as a Numba-jittable CSR matrix from `Neumann`:

In [ ]:
pd.nodal_adjacency_matrix(frmt='csr')

## Pseudo Peripheral Nodes

In [ ]:
from neumann.topology.graph import pseudo_peripheral_nodes
csr = pd.nodal_adjacency_matrix(frmt='csr')
ppn = pseudo_peripheral_nodes(csr)

## Rooted Level Structure

In [ ]:
from neumann.topology.graph import rooted_level_structure
rls = rooted_level_structure(csr)

## K-Nearest-Neighbours with ``Scipy`` or ``SkLearn``

In [15]:
from polymesh.grid import Grid

size = 80, 60, 20
shape = 10, 8, 4
grid = Grid(size=size, shape=shape, eshape='H8')
grid.k_nearest_cell_neighbours(k=3, knn_options=dict(max_distance=10.0))[:5]

array([[0, 1, 4],
       [1, 2, 0],
       [2, 3, 1],
       [3, 2, 7],
       [4, 5, 0]], dtype=int64)